# Transmon systems dynamics

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from qutip import*
from scipy.special import eval_hermite as hpoly
import sys
sys.path.append('/Users/longnguyen/Documents/GitHub/Fluxonium_berkeley/')
from Fluxonium_hamiltonians import qchard_coupobj as coupobj
from Fluxonium_hamiltonians import qchard_transmon as transmon
from Fluxonium_hamiltonians import qchard_pshapes as pshapes
from Fluxonium_hamiltonians import qchard_evolgates as gates
import plotting_settings
import scipy.linalg

## Parameters 

In [3]:
omega1 = 5.2541 #GHz
omega2 = 5.3310
omega3 = 5.4910
alpha1 = 4.9770-omega1
alpha2 = 5.0589-omega2
alpha3 = 5.2192-omega3
nlev = 3
control1 = transmon.TransmonSimple(omega_q = omega1, alpha = alpha1, nlev = nlev)
target = transmon.TransmonSimple(omega_q = omega2, alpha = alpha2, nlev = nlev)
control2 = transmon.TransmonSimple(omega_q = omega3, alpha = alpha3, nlev = nlev)
g = 2*np.pi*0.00279
coupling_type = 'JC-rwa'

In [ ]:
#Sweep the coupling strength, charge
E_int_array = np.linspace(0, 0.1, 101)
energies_coupled_swE = np.zeros((control1.nlev * control2.nlev * target.nlev, len(E_int_array)))
nlev_show = 10

for idx, E_int in enumerate(E_int_array):
    system = coupobj.CoupledObjects(control1, target, control2,
    [control1, target, E_int, coupling_type],
    [control2, target, E_int, coupling_type])
    energies_coupled_swE[:,idx] = system.levels()

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize = [12,6])
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=None)

#Transition freqs
for idx in range(0, nlev_show):
    ax1.plot(E_int_array, energies_coupled_swE[idx, :]-energies_coupled_swE[0, :], linewidth = 2.0)
ax1.set_xlabel('$J_C$ (GHz)')
ax1.set_ylabel('Eigenenergies (GHz)')
ax1.set_xlim([E_int_array[0], E_int_array[-1]])
ax1.set_ylim([-0.2,15])

#State labeling
for idx in range(0, nlev_show):
    ax2.plot(E_int_array, 0.1 * idx+(energies_coupled_swE[idx, :]-energies_coupled_swE[idx, 0]), linewidth = 2.0)
    label = (r'$|$' + system.level_label(idx, label_format='str') + r'$\rangle$')
    ax2.text(0, 0.02 + 0.1 * idx, label, fontsize=12)
ax2.set_xlabel('$J_C$ (GHz)')
ax2.set_ylabel(r'$\delta f$')
ax2.set_xlim([E_int_array[0], E_int_array[-1]])

## Drive

In [6]:
system = coupobj.CoupledObjects(control1, target, control2,
    [control1, target, g, coupling_type],
    [control2, target, g, coupling_type])

#Gate parameters
T_gate = 340 #ns
transition_to_drive = ('000','010')
drive_amplitude_factor = 1
delta_omega_d = 0.0

#pulse shape
shape = 'gauss'
sigma = 0.25 #in unit of T_gate
nlev_q = 5

#Drive amplitude and frequency
level1, level2 = transition_to_drive[0], transition_to_drive[1]
omega_d = abs(system.freq(level1, level2)) + delta_omega_d
matr_el = np.abs(system.a_ij(control1, level1, level2) + system.adag_ij(control1, level1, level2)
                 + system.a_ij(control2, level1, level2) + system.adag_ij(control2, level1, level2))
epsilon = drive_amplitude_factor / abs(matr_el)

#Evolve gate
t_points = np.linspace(0, T_gate, 2 * int(T_gate) + 1)
H_drive = epsilon * (system.a(0) + system.a(0).dag() + system.a(1) + system.a(1).dag())
comp_space = ('|000>', '|100>', '|010>', '|001>', '|110>', '|011>', '|101>', '|111>')
U_t = gates.evolution_subspace_microwave(
        system.H(), H_drive, subspace_states = comp_space, t_points=t_points, 
        T_gate=T_gate, shape=shape, sigma=sigma, omega_d=omega_d)

AttributeError: 'str' object has no attribute 'isket'